In [1]:
#!pip install transformers[torch]

In [2]:
from datasets import Dataset
import numpy as np
import pickle
import transformers
from tqdm import tqdm

In [3]:
import torch
torch.cuda.is_available()

True

In [4]:
with open("../data/test_dset_prev5.pkl", 'rb') as f:
    test_dset = pickle.load(f)

with open("../data/train_dset_prev5.pkl", "rb") as f:
    train_dset = pickle.load(f)

In [5]:
from transformers import AutoModelForSequenceClassification, TrainingArguments, Trainer

#Select a model checkpoint. For my purposes, training on 3080 Ti at home with 12GB RAM
#Will select smaller model because I don't want to overburden my GPU =)
#Models
#Checkpoint name	Num layers	Num parameters
#esm2_t48_15B_UR50D	48	15B
#esm2_t36_3B_UR50D	36	3B
#esm2_t33_650M_UR50D	33	650M
#esm2_t30_150M_UR50D	30	150M
#esm2_t12_35M_UR50D	12	35M
#esm2_t6_8M_UR50D	6	8M

model_checkpoint = "facebook/esm2_t12_35M_UR50D"

num_labels = 2
model = AutoModelForSequenceClassification.from_pretrained(model_checkpoint, num_labels=num_labels)

2024-07-28 03:51:25.331997: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2024-07-28 03:51:25.767289: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-07-28 03:51:26.790737: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
Some weights of EsmForSequenceClassification were not initialized from the model checkpoint at facebook/esm2_t12_35M_UR50D and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [6]:
model_name = model_checkpoint.split("/")[-1]
batch_size = 8

args = TrainingArguments(
    f"{model_name}_MHCI_classification",
    eval_strategy = "epoch",
    save_strategy = "epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=3,
    weight_decay=0.01,
    load_best_model_at_end=True,
    metric_for_best_model="accuracy",
    push_to_hub=False,
)

In [7]:
from evaluate import load
import numpy as np

metric = load("accuracy")

def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    return metric.compute(predictions=predictions, references=labels)

In [8]:
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

trainer = Trainer(
    model,
    args,
    train_dataset=train_dset,
    eval_dataset=test_dset,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
)

In [9]:
trainer.train()

Epoch,Training Loss,Validation Loss,Accuracy
1,0.125600,0.135535,0.963238
2,0.102200,0.118114,0.971784
3,0.067800,0.108424,0.974683


TrainOutput(global_step=310935, training_loss=0.13327288883038757, metrics={'train_runtime': 35423.7353, 'train_samples_per_second': 70.221, 'train_steps_per_second': 8.778, 'total_flos': 1.896651353114737e+17, 'train_loss': 0.13327288883038757, 'epoch': 3.0})

In [ ]:
with open("../data/trained_model.pkl", 'wb') as f: 
    pickle.dump(trainer, f)